# nvImageCodec batch processing examples

In [ ]:
import os
import os.path
from matplotlib import pyplot as plt
from nvidia import nvimgcodec

Setting resource folder

In [ ]:
resources_dir = os.getenv("PYNVIMGCODEC_EXAMPLES_RESOURCES_DIR", "../assets/images/")

Load images

In [ ]:
image_paths = [resources_dir + "cat-1046544_640.jp2", resources_dir + "tabby_tiger_cat.jpg",
               resources_dir + "Weimaraner.bmp"]
batch_size = len(image_paths)

data_list = []
for p in image_paths:
    with open(p, 'rb') as in_file:
        data = in_file.read()
        data_list.append(data)

Create decoder

In [ ]:
dec = nvimgcodec.Decoder()

Decode data

In [ ]:
image_list = dec.decode(data_list)

Decode files

In [ ]:
image_list = dec.read(image_paths)

Show decoded images

In [ ]:
for img in image_list:
    print(img.__cuda_array_interface__)
    plt.figure()
    plt.imshow(img.cpu())

In [ ]:
enc = nvimgcodec.Encoder()

In [ ]:
encoded_imgs = enc.encode(image_list, ".jpg")

Write data to file

In [ ]:
i = 0
for enc_img in encoded_imgs:
    out_file_name = os.path.splitext(os.path.basename(image_paths[i]))[0]+ "_out.jpg"
    with open(out_file_name, "wb") as out_file:
        out_file.write(enc_img)
    i+=1

Encode and write images to file in one write function

In [ ]:
out_file_names = [os.path.splitext(os.path.basename(p))[0] + "_out.jpg" for p in image_paths]
enc.write(out_file_names, image_list, ".jpg")

if we skip codec argument, files will be encoded to codec deducted based on file extension

In [ ]:
enc.write(out_file_names, image_list)

Decode all files in directory

In [ ]:
import glob
import cupy as cp
image_paths = glob.glob(resources_dir + '*.*')
batch_size = 5
file_name_batches = [image_paths[i: i + batch_size] for i in range(0, len(image_paths), batch_size)] 
for file_name_batch in file_name_batches:
    #Alternatively, load data from each file and pass data_list to decode
    """data_list = []
    for path in file_name_batch:
        with open(path, "rb") as in_file:
            data_list.append(in_file.read())
    image_list = dec.decode(data_list)"""

    image_list = dec.read(file_name_batch)

    for img in image_list:
        h_img = img.cpu()
        plt.figure()
        plt.imshow(h_img)
